In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import cv2
import numpy as np
import tensorflow as tf
# !pip install -U keras
# !pip install tensorflow-gpu==2.1.0rc0

In [3]:
import math
import random
import keras
from keras.layers import *
from keras.models import Sequential
from keras import Model
from keras import backend as K  
from keras.regularizers import l2
from keras.callbacks import EarlyStopping,ModelCheckpoint

from sklearn.model_selection import LeaveOneOut
from sklearn import metrics

Using TensorFlow backend.


In [4]:
def get_conv_block(input_layer,nFilters,size):
    conv1 = Conv2D(nFilters, size, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(input_layer)
    bn1 = BatchNormalization()(conv1)
    conv2 = Conv2D(nFilters, size, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(bn1)
    bn2 = BatchNormalization()(conv2)
    return bn2
    
def get_UNET(input_layer,nFilters): 

    block1 = get_conv_block(input_layer,nFilters,3)
    mp1 = MaxPooling2D(pool_size=(2, 2))(block1)
    dr1 = Dropout(0.1)(mp1)
   
    block2 = get_conv_block(dr1,nFilters*2,3)
    mp2 = MaxPooling2D(pool_size=(2, 2))(block2)
    dr2 = Dropout(0.1)(mp2)

    block3 = get_conv_block(dr2,nFilters*4,3)
    mp3 = MaxPooling2D(pool_size=(2, 2))(block3)
    dr3 = Dropout(0.1)(mp3)

    block4 = get_conv_block(dr3,nFilters*8,3)
    mp4 = MaxPooling2D(pool_size=(2, 2))(block4)
    dr4 = Dropout(0.1)(mp4)

    conv5 = Conv2D(nFilters*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(dr4)
    conv5 = Conv2D(nFilters*16, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(conv5)

    up1 = Conv2DTranspose(nFilters*8,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(conv5)
    cat1 = concatenate([block4, up1])
    dr1 = Dropout(0.1)(cat1)
    block5 = get_conv_block(dr1,nFilters*8,3)

    up2 = Conv2DTranspose(nFilters*4,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block5)
    cat2 = concatenate([block3, up2])
    dr2 = Dropout(0.1)(cat2)
    block6 = get_conv_block(dr2,nFilters*4,3)
    
    up3 = Conv2DTranspose(nFilters*2,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block6)
    cat3 = concatenate([block2, up3])
    dr3 = Dropout(0.1)(cat3)
    block7 = get_conv_block(dr3,nFilters*2,3)
    
    up4 = Conv2DTranspose(nFilters,(3,3),strides =(2,2),activation='relu',padding='same',kernel_initializer = 'he_normal')(block7)
    cat4 = concatenate([block1, up4])
    dr4 = Dropout(0.1)(cat4)
    block8 = get_conv_block(dr4,nFilters,3)

    conv10 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal',kernel_regularizer=l2(1e-4))(block8)
    out = Conv2D(4,(1,1), activation='softmax', padding = 'same',kernel_regularizer=l2(1e-4))(conv10)

    return out

def get_model(input_shape,nFilters):

    input_layer = Input(shape=input_shape)
    out = get_UNET(input_layer,nFilters)
    model = Model(input_layer,out)
    return model

In [5]:
PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/4 Class ER"

# Loading the data patient wise
X1 = np.load(PATH + '/ER IHC 230 Images.npy')
Y1 = np.load(PATH + '/ER IHC 230 Masks.npy')

X2 = np.load(PATH + '/ER IHC 232 Images.npy')
Y2 = np.load(PATH + '/ER IHC 232 Masks.npy')

X3 = np.load(PATH + '/ER IHC 242 Images.npy')
Y3 = np.load(PATH + '/ER IHC 242 Masks.npy')

X4 = np.load(PATH + '/ER IHC 263 Images.npy')
Y4 = np.load(PATH + '/ER IHC 263 Masks.npy')

X5 = np.load(PATH + '/ER IHC 221 Images.npy')
Y5 = np.load(PATH + '/ER IHC 221 Masks.npy')

X6 = np.load(PATH + '/ER IHC 229 Images.npy')
Y6 = np.load(PATH + '/ER IHC 229 Masks.npy')

X7 = np.load(PATH + '/ER IHC 239 Images.npy')
Y7 = np.load(PATH + '/ER IHC 239 Masks.npy')

X8 = np.load(PATH + '/ER IHC 246 Images.npy')
Y8 = np.load(PATH + '/ER IHC 246 Masks.npy')

X9 = np.load(PATH + '/ER IHC 252 Images.npy')
Y9 = np.load(PATH + '/ER IHC 252 Masks.npy')

In [6]:
X = [X1, X2, X3, X4, X5, X6, X7, X8, X9]
Y = [Y1, Y2, Y3, Y4, Y5, Y6, Y7, Y8, Y9]
patient_no = ['230','232','242','263','221','229','239','246','252']
size = [10,10,10,10,10,10,10,10,10]

In [7]:
fold_split = []

#test patient = 230, 232 
fold_split.append(([2,3,4,5,6,7,8],[0,1]))

# #test_patient = 239, 242
# fold_split.append(([0,1,3,4,5,7,8],[2,6]))

# #test_patient = 252,263
# fold_split.append(([0,1,2,4,5,6,7],[3,8]))

In [8]:
def convertToLabels(data):
  data[data==85]=1
  data[data==170]=2
  data[data==255]=3

def convertFromLabels(data):
  data[data==1]=85
  data[data==2]=170
  data[data==3]=255

In [9]:
def dice_coef(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + 1) / (K.sum(y_true_f) + K.sum(y_pred_f) + 1)

In [10]:
def tversky_loss_1(y_true, y_pred):
    alpha = 0.7
    beta  = 0.3
    
    ones = K.ones(K.shape(y_true))
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
  
    g0 = y_true
    g1 = ones-y_true
    
    num = K.sum(p0*g0, (0,1,2))
    den = num + alpha*K.sum(p0*g1,(0,1,2)) + beta*K.sum(p1*g0,(0,1,2))
    # 
    T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T

def tversky_loss_2(y_true, y_pred):
    alpha = 0.3
    beta  = 0.7
    
    ones = K.ones(K.shape(y_true))
    p0 = y_pred      # proba that voxels are class i
    p1 = ones-y_pred # proba that voxels are not class i
    g0 = y_true
    g1 = ones-y_true
    
    num = K.sum(p0*g0, (0,1,2))
    den = num + alpha*K.sum(p0*g1,(0,1,2)) + beta*K.sum(p1*g0,(0,1,2))
    # 
    T = K.sum(num/den) # when summing over classes, T has dynamic range [0 Ncl]
    
    Ncl = K.cast(K.shape(y_true)[-1], 'float32')
    return Ncl-T

def focal_tversky_loss_1(y_true, y_pred, gamma=0.75):
    tversky_index = tversky_loss_1(y_true, y_pred)
    return K.pow((tversky_index), gamma)
  
def focal_tversky_loss_2(y_true, y_pred, gamma=0.75):
    tversky_index = tversky_loss_2(y_true, y_pred)
    return K.pow((tversky_index), gamma)

def combined_loss(y_true, y_pred):
  return (0.2*K.categorical_crossentropy(y_true, y_pred))+(0.8*focal_tversky_loss_1(y_true, y_pred)+(0.8*focal_tversky_loss_2(y_true, y_pred)))

In [11]:
batch_size = 8
def get_batch(batch_size, X_train, Y_train): 
    size_batch = batch_size
    last_index = len(X_train) - 1
    x_train = X_train
    y_train = Y_train 
    while True:
        batch_data = [[],[]]
        for i in range(0, size_batch):
            random_index = random.randint(0, last_index)
            batch_data[0].append(x_train[random_index])
            batch_data[1].append(y_train[random_index])

        yield (np.array(batch_data[0]), np.array(batch_data[1]))     

In [12]:
MODELS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Models/4 Class/ER/UNET"

fold=0

for train_index, test_index in fold_split:
  print("TRAIN:", train_index, "TEST:", test_index, "FOLD:", fold)

  TrainX =  np.concatenate(np.array([X[i] for i in train_index]))
  TrainY =  np.concatenate(np.array([Y[i] for i in train_index]))
  TestX  =  np.concatenate(np.array([X[i] for i in test_index]))
  TestY  =  np.concatenate(np.array([Y[i] for i in test_index]))

  Train = list(zip(TrainX,TrainY))
  random.shuffle(Train)
  TrainX,TrainY = zip(*Train)
  TrainX = np.asarray(list(TrainX))
  TrainY = np.asarray(list(TrainY))

  Test = list(zip(TestX,TestY))
  random.shuffle(Test)
  TestX,TestY = zip(*Test)
  TestX = np.asarray(list(TestX))
  TestY = np.asarray(list(TestY))

  TrainX = TrainX.astype('float32')/255
  TestX = TestX.astype('float32')/255
  convertToLabels(TrainY)
  TrainY = keras.utils.to_categorical(TrainY,num_classes=4,dtype='int16')
  convertToLabels(TestY)
  TestY = keras.utils.to_categorical(TestY,num_classes=4,dtype='int16')
  ValX = TrainX[(int)(0.8*TrainX.shape[0]):]
  ValY = TrainY[(int)(0.8*TrainY.shape[0]):]
  model = get_model((240,240,3),64)

  #es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
  mc = ModelCheckpoint('Checkpoint.h5', monitor='val_loss', mode='min', verbose=1, save_best_only=True)

  optimizer=keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=0.001, decay=0.0, amsgrad=True)

  model.compile(loss=combined_loss, optimizer= optimizer , metrics=[dice_coef,'accuracy']) 
  num_epoch = 100
  datagen = get_batch(batch_size, TrainX, TrainY)
  n_points = len(TrainX)
  print('-----------fold {}--------------'.format(fold))
  history = model.fit(datagen, validation_data = [ValX, ValY],
                  epochs=num_epoch,steps_per_epoch = math.ceil(n_points / batch_size), callbacks =[mc],  shuffle =True)
  model.save(MODELS_PATH + '/UNET_ER_'+ str(fold) +'.h5')
  fold = fold + 1

TRAIN: [2, 3, 4, 5, 6, 7, 8] TEST: [0, 1] FOLD: 0
-----------fold 0--------------
Epoch 1/100
420/420 [==============================] - 193s 459ms/step - loss: 4.3357 - dice_coef: 0.8616 - accuracy: 0.8971 - val_loss: 4.0433 - val_dice_coef: 0.9429 - val_accuracy: 0.9470

Epoch 00001: val_loss improved from inf to 4.04327, saving model to Checkpoint.h5
Epoch 2/100
420/420 [==============================] - 179s 427ms/step - loss: 3.9148 - dice_coef: 0.9415 - accuracy: 0.9460 - val_loss: 3.8549 - val_dice_coef: 0.9492 - val_accuracy: 0.9534

Epoch 00002: val_loss improved from 4.04327 to 3.85485, saving model to Checkpoint.h5
Epoch 3/100
420/420 [==============================] - 179s 427ms/step - loss: 3.6973 - dice_coef: 0.9514 - accuracy: 0.9562 - val_loss: 3.6530 - val_dice_coef: 0.9529 - val_accuracy: 0.9571

Epoch 00003: val_loss improved from 3.85485 to 3.65300, saving model to Checkpoint.h5
Epoch 4/100
420/420 [==============================] - 179s 427ms/step - loss: 3.5631 - 

In [13]:
def stitchMaskPatches(pieces):
  k = 0
  reconstructed_img = np.ones([1440,1920])
  for r in range(6):
    row = r * 240
    for c in range(8):
      col = c * 240
      reconstructed_img[row:row+240,col:col+240] = pieces[k]
      k = k + 1
  return reconstructed_img


def stitchImagePatches(pieces):
  k = 0
  reconstructed_img = np.ones([1440,1920,3])
  for r in range(6):
    row = r * 240
    for c in range(8):
      col = c * 240
      reconstructed_img[row:row+240,col:col+240,:] = pieces[k]
      k = k + 1
  return reconstructed_img

In [14]:
def saveNumpyOutput(mask, Patient_array,Patient_length,title,folder):
  idx = 0
  for i in range(len(Patient_length)):
    temp = []
    for j in range(Patient_length[i]):
      final_output = mask[idx:idx+48]
      idx = idx + 48
      final_output = stitchMaskPatches(final_output)
      temp.append(final_output)
    final_output = np.asarray(temp)
    np.save(folder + title + Patient_array[i], final_output)

In [15]:
MODELS_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Models/4 Class/ER/UNET/"
model_names = os.listdir(MODELS_PATH)
print(model_names)

['UNET_ER_0.h5']


In [16]:
SAVE_PATH = "/content/drive/My Drive/Breast Cancer Treatment/Numpy Arrays/Predicted Output/ER/UNET/"

In [17]:
# Axis 0 = Folds
# Axis 1 = Class (Strong Intermediate,Weak,Background)
# Axis 2 = Evaluation Technique (Precision, Recall, Dice coefficient)
pixEvaluationTrain = np.empty((0,4,3))
pixEvaluationTest = np.empty((0,4,3))
score = np.zeros((1,4,3))

In [19]:
fold = 0

for train_index, test_index in fold_split:
  print("FOLD:",fold,"\tTRAIN:", train_index, "TEST:", test_index)
  # Splitting Train and Test data
  TrainX =  np.concatenate(np.array([X[i] for i in train_index]))
  TrainY =  np.concatenate(np.array([Y[i] for i in train_index]))
  TestX  =  np.concatenate(np.array([X[i] for i in test_index]))
  TestY  =  np.concatenate(np.array([Y[i] for i in test_index]))

  train_no = [patient_no[i] for i in train_index]
  train_size = [size[i] for i in train_index]

  test_no = [patient_no[i] for i in test_index]
  test_size = [size[i] for i in test_index]

  TrainX = TrainX.astype('float32')/255
  TestX = TestX.astype('float32')/255

  convertToLabels(TrainY)
  convertToLabels(TestY)

  # Loading corrosponding fold of model
  model = keras.models.load_model(MODELS_PATH + model_names[fold],custom_objects={ 'combined_loss': combined_loss, 'dice_coef': dice_coef })

  # Predicting results using model
  trainResult = model.predict(TrainX, batch_size=8)
  testResult = model.predict(TestX,batch_size=8)

  trainResult = np.argmax(trainResult,axis=-1)
  testResult = np.argmax(testResult,axis=-1)

  # Obtaining pixel-wise results: Precision recall and dice coefficient
  # For TRAIN
  y_true = np.reshape(TrainY,(-1))
  y_pred = np.reshape(trainResult,(-1))

  prec   = metrics.precision_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  recall = metrics.recall_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  dice = 2*prec*recall/(prec+recall)

  # Precision Recall Dice for each class
  for i in range(4):
    score[0][i][0] = prec[3-i]
    score[0][i][1] = recall[3-i]
    score[0][i][2] = dice[3-i]

  pixEvaluationTrain = np.append(pixEvaluationTrain,score,axis=0)
 
  # For TEST
  y_true = np.reshape(TestY,(-1))
  y_pred = np.reshape(testResult,(-1))

  prec   = metrics.precision_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  recall = metrics.recall_score(y_true,y_pred,labels=[0,1,2,3],average=None,zero_division=1)
  dice = 2*prec*recall/(prec+recall)

  # Precision Recall Dice for each class
  for i in range(4):
    score[0][i][0] = prec[3-i]
    score[0][i][1] = recall[3-i]
    score[0][i][2] = dice[3-i]

  pixEvaluationTest = np.append(pixEvaluationTest,score,axis=0)

  # Converting predicted labels to required output format
  
  convertFromLabels(trainResult)
  convertFromLabels(testResult)

  # Saving numpy arrays
  path = os.path.join(SAVE_PATH,"Fold " + str(fold))
  os.mkdir(path)
  os.mkdir(path + '/Train')
  os.mkdir(path + '/Test')

  saveNumpyOutput(trainResult, train_no, train_size,"/Train/",path)
  saveNumpyOutput(testResult, test_no, test_size,"/Test/",path)

  fold +=1

FOLD: 0 	TRAIN: [2, 3, 4, 5, 6, 7, 8] TEST: [0, 1]


In [20]:
print(pixEvaluationTrain.shape)
print(pixEvaluationTest.shape)
print(pixEvaluationTest)

(1, 4, 3)
(1, 4, 3)
[[[0.89726281 0.29532795 0.44438846]
  [0.39303243 0.59722515 0.47407635]
  [0.31631779 0.43318254 0.36563918]
  [0.97582288 0.98184625 0.9788253 ]]]


In [21]:
np.save(SAVE_PATH + "Pixel-wise Metrics Train.npy", pixEvaluationTrain)
np.save(SAVE_PATH + "Pixel-wise Metrics Test.npy", pixEvaluationTest)

# **DISPLAY**

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


plt.figure(figsize=(20,20))
id = 48   # enter between 0- 50 since there are 5 patients with 10 images each

id = id * 48
final_input = TestX[id:id+48]
Mask_input = TestGT[id:id+48]
final_output = testResult[id:id+48]

final_input = stitchImagePatches(final_input)
Mask_input =  stitchMaskPatches(Mask_input)
final_output = stitchMaskPatches(final_output)

print(final_input.shape)
print(final_output.shape)
print(Mask_input.shape)

print(np.unique(final_output))
copy1  = np.copy(final_input)
copy2 = copy1.astype('float32')*255
copy2 = copy2.astype('uint8')
final_input = np.reshape(copy2,(1440, 1920,3))
final_input = cv2.cvtColor(final_input,cv2.COLOR_BGR2RGB)

plt.subplot(131).imshow(final_input)
plt.subplot(132).imshow(Mask_input,'gray')
plt.subplot(133).imshow(final_output,'gray')

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow


plt.figure(figsize=(20,20))
id = 25  # enter between 0- 40 since there are 4 patients with 10 images each

id = id * 48
final_input = TrainX[id:id+48]
Mask_input = TrainGT[id:id+48]
final_output = trainResult[id:id+48]

final_input = stitchImagePatches(final_input)
Mask_input =  stitchMaskPatches(Mask_input)
final_output = stitchMaskPatches(final_output)

print(final_input.shape)
print(final_output.shape)
print(Mask_input.shape)

print(np.unique(final_output))
copy1  = np.copy(final_input)
copy2 = copy1.astype('float32')*255
copy2 = copy2.astype('uint8')
final_input = np.reshape(copy2,(1440, 1920,3))
final_input = cv2.cvtColor(final_input,cv2.COLOR_BGR2RGB)

plt.subplot(131).imshow(final_input)
plt.subplot(132).imshow(Mask_input,'gray')
plt.subplot(133).imshow(final_output,'gray')